In [2]:
import zipfile
with zipfile.ZipFile('ml-latest-small.zip', 'r') as zip_ref:
    zip_ref.extractall('data')

In [2]:
# import the dataset
import pandas as pd
movies_df = pd.read_csv('data/ml-latest-small/movies.csv')
ratings_df = pd.read_csv('data/ml-latest-small/ratings.csv')

In [3]:
print('The dimensions of movies dataframe are:', movies_df.shape,'\nThe dimensions of ratings dataframe are:', ratings_df.shape)

The dimensions of movies dataframe are: (9742, 3) 
The dimensions of ratings dataframe are: (100836, 4)


In [4]:
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [6]:
# Movie ID to movie name mapping
movie_names = movies_df.set_index('movieId')['title'].to_dict()
n_users = len(ratings_df.userId.unique())
n_items = len(ratings_df.movieId.unique())
print("Number of unique users:", n_users)
print("Number of unique movies:", n_items)
print("The full rating matrix will have:", n_users*n_items, 'elements.')
print('----------')
print("Number of ratings:", len(ratings_df))
print("Therefore: ", len(ratings_df) / (n_users*n_items) * 100, '% of the matrix is filled.')
print("We have an incredibly sparse matrix to work with here.")
print("And... as you can imagine, as the number of users and products grow, the number of elements will increase by n*2")
print("You are going to need a lot of memory to work with global scale... storing a full matrix in memory would be a challenge.")
print("One advantage here is that matrix factorization can realize the rating matrix implicitly, thus we don't need all the data")

Number of unique users: 610
Number of unique movies: 9724
The full rating matrix will have: 5931640 elements.
----------
Number of ratings: 100836
Therefore:  1.6999683055613624 % of the matrix is filled.
We have an incredibly sparse matrix to work with here.
And... as you can imagine, as the number of users and products grow, the number of elements will increase by n*2
You are going to need a lot of memory to work with global scale... storing a full matrix in memory would be a challenge.
One advantage here is that matrix factorization can realize the rating matrix implicitly, thus we don't need all the data


In [12]:
import torch
import numpy as np
from torch.autograd import Variable
from tqdm import tqdm_notebook as tqdm

class MatrixFactorization(torch.nn.Module):
    def __init__(self, n_users, n_items, n_factors=20):
        super().__init__()
        # create user embeddings
        self.user_factors = torch.nn.Embedding(n_users, n_factors) # think of this as a lookup table for the input.
        # create item embeddings
        self.item_factors = torch.nn.Embedding(n_items, n_factors) # think of this as a lookup table for the input.
        self.user_factors.weight.data.uniform_(0, 0.05)
        self.item_factors.weight.data.uniform_(0, 0.05)
        
    def forward(self, data):
        # matrix multiplication
        users, items = data[:,0], data[:,1]
        return (self.user_factors(users)*self.item_factors(items)).sum(1)
    # def forward(self, user, item):
    # 	# matrix multiplication
    #     return (self.user_factors(user)*self.item_factors(item)).sum(1)
    
    def predict(self, user, item):
        return self.forward(user, item)

In [58]:
# Creating the dataloader (necessary for PyTorch)
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader # package that helps transform your data to machine learning readiness

# Note: This isn't 'good' practice, in a MLops sense but we'll roll with this since the data is already loaded in memory.
class Loader(Dataset):
    def __init__(self):
        self.ratings = ratings_df.copy()
        
        # Extract all user IDs and movie IDs
        users = ratings_df.userId.unique()
        movies = ratings_df.movieId.unique()
        
        #--- Producing new continuous IDs for users and movies ---
        
        # Unique values : index
        self.userid2idx = {o:i for i,o in enumerate(users)}
        self.movieid2idx = {o:i for i,o in enumerate(movies)}
        
        # Obtained continuous ID for users and movies
        self.idx2userid = {i:o for o,i in self.userid2idx.items()}
        self.idx2movieid = {i:o for o,i in self.movieid2idx.items()}
        
        # return the id from the indexed values as noted in the lambda function down below.
        self.ratings.movieId = ratings_df.movieId.apply(lambda x: self.movieid2idx[x])
        self.ratings.userId = ratings_df.userId.apply(lambda x: self.userid2idx[x])
        
        
        self.x = self.ratings.drop(['rating', 'timestamp'], axis=1).values
        self.y = self.ratings['rating'].values
        self.x, self.y = torch.tensor(self.x), torch.tensor(self.y) # Transforms the data to tensors (ready for torch models.)

    def __getitem__(self, index):
        return (self.x[index], self.y[index])

    def __len__(self):
        return len(self.ratings)

In [60]:
num_epochs = 128
cuda = torch.cuda.is_available()

print("Is running on GPU:", cuda)

model = MatrixFactorization(n_users, n_items, n_factors=8)
print(model)
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)
# GPU enable if you have a GPU...
if cuda:
    model = model.cuda()

# MSE loss
loss_fn = torch.nn.MSELoss()

# ADAM optimizier
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Train data
train_set = Loader()
train_loader = DataLoader(train_set, 128, shuffle=True)

In [61]:
for it in tqdm(range(num_epochs)):
    losses = []
    for x, y in train_loader:
         if cuda:
            x, y = x.cuda(), y.cuda()
            optimizer.zero_grad()
            outputs = model(x)
            loss = loss_fn(outputs.squeeze(), y.type(torch.float32))
            losses.append(loss.item())
            loss.backward()
            optimizer.step()
    print("iter #{}".format(it), "Loss:", sum(losses) / len(losses))

C:\Users\User\AppData\Local\Temp\ipykernel_27300\59576823.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for it in tqdm(range(num_epochs)):


  0%|          | 0/128 [00:00<?, ?it/s]

iter #0 Loss: 11.065572240025864
iter #1 Loss: 4.740930120049394
iter #2 Loss: 2.4751377284224265
iter #3 Loss: 1.721292265931967
iter #4 Loss: 1.3459854307513552
iter #5 Loss: 1.1283378847661962
iter #6 Loss: 0.9913622207447962
iter #7 Loss: 0.9004392309237252
iter #8 Loss: 0.837152705428564
iter #9 Loss: 0.7922112596443462
iter #10 Loss: 0.759573407849382
iter #11 Loss: 0.7344414004063243
iter #12 Loss: 0.716166060512441
iter #13 Loss: 0.7015218915143594
iter #14 Loss: 0.6901392995327862
iter #15 Loss: 0.6819645296498603
iter #16 Loss: 0.6748056159104188
iter #17 Loss: 0.6696640322656195
iter #18 Loss: 0.6659144207759558
iter #19 Loss: 0.6629100435125042
iter #20 Loss: 0.6606351872386061
iter #21 Loss: 0.6590980654818758
iter #22 Loss: 0.6578387821537589
iter #23 Loss: 0.6566180370105099
iter #24 Loss: 0.6557985077503369
iter #25 Loss: 0.6551122055743551
iter #26 Loss: 0.6544164072362905
iter #27 Loss: 0.6533871745019395
iter #28 Loss: 0.6529030295722376
iter #29 Loss: 0.651480603452

In [62]:
# By training the model, we will have tuned latent factors for movies and users.
c = 0
uw = 0
iw = 0 
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)
        if c == 0:
          uw = param.data
          c +=1
        else:
          iw = param.data
        #print('param_data', param_data)

user_factors.weight tensor([[ 1.0572,  0.5450,  1.2162,  ...,  1.5213,  1.3887,  1.4341],
        [ 1.4839,  1.7246,  1.2278,  ...,  2.2224,  1.3568,  0.7843],
        [ 2.2968,  1.4438, -1.4461,  ...,  1.4732,  1.1357,  0.9361],
        ...,
        [ 1.0330,  0.3806,  2.7327,  ...,  0.0598,  0.1277,  0.5792],
        [ 0.7412,  0.5252,  1.5370,  ...,  1.2113,  0.9574,  0.8363],
        [ 1.9528,  1.1362,  0.6658,  ...,  1.0170,  1.2118,  0.8589]],
       device='cuda:0')
item_factors.weight tensor([[ 0.3450,  0.4665,  0.3773,  ...,  0.6724,  0.6289,  0.4812],
        [ 0.4300,  0.3455, -0.1292,  ...,  1.0156,  0.4395,  0.3091],
        [ 0.6390,  0.4383,  0.5157,  ...,  0.2421,  0.1789,  0.7749],
        ...,
        [ 0.3313,  0.3414,  0.3595,  ...,  0.3497,  0.3414,  0.3693],
        [ 0.3843,  0.3837,  0.3884,  ...,  0.3727,  0.3825,  0.3909],
        [ 0.3784,  0.4034,  0.3871,  ...,  0.4113,  0.3740,  0.3904]],
       device='cuda:0')


In [63]:
trained_movie_embeddings = model.item_factors.weight.data.cpu().numpy()

In [64]:
len(trained_movie_embeddings) # unique movie factor weights

9724

In [65]:
from sklearn.cluster import KMeans
# Fit the clusters based on the movie weights
kmeans = KMeans(n_clusters=10, random_state=0).fit(trained_movie_embeddings)

c:\Users\User\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [66]:
'''It can be seen here that the movies that are in the same cluster tend to have
similar genres. Also note that the algorithm is unfamiliar with the movie name
and only obtained the relationships by looking at the numbers representing how
users have responded to the movie selections.'''
for cluster in range(10):
    print("Cluster #{}".format(cluster))
    movs = []
    # Find movie indices belonging to the current cluster
    for movidx in np.where(kmeans.labels_ == cluster)[0]:
        movid = train_set.idx2movieid[movidx]
        # Check how many ratings this movie has
        rat_count = len(ratings_df.loc[ratings_df['movieId'] == movid])
        movs.append((movie_names[movid], rat_count))
    # Sort movies by rating count in descending order and print top 10
    for mov in sorted(movs, key=lambda tup: tup[1], reverse=True)[:10]:
        print("\t", mov[0])

Cluster #0
	 True Lies (1994)
	 Back to the Future (1985)
	 Die Hard: With a Vengeance (1995)
	 Batman Forever (1995)
	 GoldenEye (1995)
	 Ghost (1990)
	 Firm, The (1993)
	 Armageddon (1998)
	 Back to the Future Part III (1990)
	 Back to the Future Part II (1989)
Cluster #1
	 Forrest Gump (1994)
	 Shawshank Redemption, The (1994)
	 Silence of the Lambs, The (1991)
	 Star Wars: Episode IV - A New Hope (1977)
	 Braveheart (1995)
	 Schindler's List (1993)
	 Fight Club (1999)
	 Star Wars: Episode V - The Empire Strikes Back (1980)
	 Usual Suspects, The (1995)
	 Lord of the Rings: The Fellowship of the Ring, The (2001)
Cluster #2
	 Ace Ventura: Pet Detective (1994)
	 Pirates of the Caribbean: The Curse of the Black Pearl (2003)
	 Star Wars: Episode I - The Phantom Menace (1999)
	 X-Men (2000)
	 Twister (1996)
	 Matrix Reloaded, The (2003)
	 Natural Born Killers (1994)
	 Star Wars: Episode II - Attack of the Clones (2002)
	 Shrek 2 (2004)
	 Star Trek: First Contact (1996)
Cluster #3
	 Mrs. D